In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import tree

import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import torch.nn as nn
from mpl_toolkits.mplot3d import Axes3D  
import plotly.graph_objects as go
from scipy.stats import gaussian_kde
import matplotlib.patches as mpatches
import matplotlib.dates as mdates
from matplotlib import pyplot as plt
from scipy import stats
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.colors as pc
from sklearn.ensemble import IsolationForest



# Dataset overview

In [ ]:
data = pd.read_csv('./Dataset_dumarey/original_csv/Complete_not_cleaned.csv')

In [ ]:
data.head()


,vin,timestamp,end_time,odo,end_odo,soc,end_soc,event,charge_mode
0,-8.99632E+l8,01/01/2021 20:31,01/01/2021 20:38,4436640625,443946875,564.0,549.0,trip,NaN
1,-8.99632E+ 18,01/01/202120:39,02/01/202115:13,443946875,443946875,549.0,100.0,charge,240
2,-8.99632E+18,02/01/202115:13,02/01/202115:21,443946875,"444,250",100.0,100.0,trip,NaN
3,-8.99632E+18,02/01/202115:21,02/01/202117:46,"444,250","445,7125",100.0,945.0,trip,NaN
4,-8.99632E+18,02/01/202117:46,02/01/202117:53,"445,7125","446,145",945.0,945.0,trip,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   vin          444 non-null    object 
 1   timestamp    444 non-null    object 
 2   end_time     444 non-null    object 
 3   odo          444 non-null    object 
 4   end_odo      444 non-null    object 
 5   soc          441 non-null    float64
 6   end_soc      443 non-null    float64
 7   event        444 non-null    object 
 8   charge_mode  85 non-null     object 
dtypes: float64(2), object(7)
memory usage: 31.3+ KB


# Data Cleaning


In [ ]:
def substitute_comma(ts):
    if pd.isna(ts):
        return ts 
    ts = str(ts).strip()
    ts = ts.replace(",", ".")     
    ts = ts.replace(", ", ".")
    ts = ts.replace(" , ", ".")    
    ts = ts.replace(" ,", ".")       
    ts = ts.replace(". ", ".")    
    ts = ts.replace(" .", ".") 
    ts = ts.replace(" . ", ".")       
    return ts 
def remove_comma_odo(ts):
    if pd.isna(ts):
        return ts 
    ts = str(ts).strip()
    ts = ts.replace(",", "")         
    ts = ts.replace(", ", "")        
    ts = ts.replace(" ,", "")     
    return ts 
def remove_comma(ts):
    if pd.isna(ts):
        return ts 
    ts = str(ts).strip()
    ts = ts.replace(",", "")         
    ts = ts.replace(". ", "")        
    ts = ts.replace(".", "")     
    return ts 

def remove_spaces(ts):
    if pd.isna(ts):
        return ts 
    ts = str(ts).strip()
    ts = ts.replace(" ", "")         
    ts = ts.replace("  ", "")        
    ts = ts.replace("   ", "")     
    return ts
def fix_timestamp_format(ts):
    if pd.isna(ts):
        return ts
    
    ts = str(ts).replace(" ", "")  
    
    if len(ts) >= 15:  # es. 27/04/202121:27
        return ts[:10] + ' ' + ts[10:]
    else:
        return ts
def fix_soc (ts):
    if ts > 100.0:
        return ts / 10
    else:
        return ts
def fix_odo(ts):
    if pd.isna(ts):
        return ts
    try:
        ts_str = str(int(ts))
        if len(ts_str) <= 4:
            return int(ts_str) 
        return float(ts_str[:4] + '.' + ts_str[4:])
    except:
        return ts
def fix_vin (ts):
    if pd.isna(ts):
        return ts 
    ts = str(ts)
    if ts.startswith('-8.9'):
        return '1.0'
    elif ts.startswith('-2.4'):
        return '2.0'
    elif ts.startswith('·2.4'):
        return '2.0'
    else:
        return ts


#### Fixing marks

In [ ]:
data['odo'] = data['odo'].apply(remove_comma_odo)
data['end_odo'] = data['end_odo'].apply(remove_comma_odo)
data['soc'] = data['soc'].apply(substitute_comma)
data['end_soc'] = data['end_soc'].apply(substitute_comma)


In [ ]:
data.head()

,vin,timestamp,end_time,odo,end_odo,soc,end_soc,event,charge_mode
0,-8.99632E+l8,01/01/2021 20:31,01/01/2021 20:38,4436640625,443946875,564.0,549.0,trip,NaN
1,-8.99632E+ 18,01/01/202120:39,02/01/202115:13,443946875,443946875,549.0,100.0,charge,240
2,-8.99632E+18,02/01/202115:13,02/01/202115:21,443946875,444250,100.0,100.0,trip,NaN
3,-8.99632E+18,02/01/202115:21,02/01/202117:46,444250,4457125,100.0,945.0,trip,NaN
4,-8.99632E+18,02/01/202117:46,02/01/202117:53,4457125,446145,945.0,945.0,trip,NaN


In [ ]:
cols_to_float = ['odo', 'end_odo', 'soc', 'end_soc']
for col in cols_to_float:
    data[col] = pd.to_numeric(data[col], errors='coerce')

#### Fixing timestamp format

In [ ]:
data['timestamp'] = data['timestamp'].apply(fix_timestamp_format)
data['end_time'] = data['end_time'].apply(fix_timestamp_format) 
data['timestamp'] = pd.to_datetime(data['timestamp'], errors='coerce', dayfirst=True)
data['end_time'] = pd.to_datetime(data['end_time'], errors='coerce', dayfirst=True)

In [ ]:
data['soc'] = data['soc'].apply(fix_soc)
data['end_soc'] = data['end_soc'].apply(fix_soc)
#data['odo'] = data['odo'].apply(fix_odo)
#data['end_odo'] = data['end_odo'].apply(fix_odo) #because it's ok for odo <= 9999


In [ ]:
data.head()

,vin,timestamp,end_time,odo,end_odo,soc,end_soc,event,charge_mode
0,-8.99632E+l8,2021-01-01 20:31:00,2021-01-01 20:38:00,4436.640625,4439.46875,56.4,54.9,trip,NaN
1,-8.99632E+ 18,2021-01-01 20:39:00,2021-01-02 15:13:00,4439.468750,4439.46875,54.9,100.0,charge,240
2,-8.99632E+18,2021-01-02 15:13:00,2021-01-02 15:21:00,4439.468750,4442.50000,100.0,100.0,trip,NaN
3,-8.99632E+18,2021-01-02 15:21:00,2021-01-02 17:46:00,4442.500000,4457.12500,100.0,94.5,trip,NaN
4,-8.99632E+18,2021-01-02 17:46:00,2021-01-02 17:53:00,4457.125000,4461.45000,94.5,94.5,trip,NaN


In [ ]:
cols_to_convert = ['event']
for col in cols_to_convert:
    data[col] = data[col].astype('category')

data['charge_mode'] = data['charge_mode'].apply(remove_spaces)
data['charge_mode'] = data['charge_mode'].replace('', '0')
data['charge_mode'] = data['charge_mode'].fillna('0')
data['charge_mode'] = data['charge_mode'].astype('category')
if '0' not in data['charge_mode'].cat.categories:
    data['charge_mode'] = data['charge_mode'].cat.add_categories('0')
data['vin'] = data['vin'].apply(fix_vin)
data['vin'] = data['vin'].astype('float64')

In [ ]:
'''def original_vin (ts):
    if pd.isna(ts):
        return ts 
    ts = str(ts)
    if ts.startswith('1.0'):
        return '-8.99632E+18'
    elif ts.startswith('2.0'):
        return '-2.47174E+18'
    else:
        return ts
'''
None

In [ ]:
#data_to_csv = data.copy()
#data_to_csv['vin'] = data_to_csv['vin'].apply(original_vin)

In [ ]:
#data_to_csv.to_csv('./Dataset_dumarey/preprocessed_dataset/dataset_original.csv', index=False)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   vin          444 non-null    float64       
 1   timestamp    444 non-null    datetime64[ns]
 2   end_time     444 non-null    datetime64[ns]
 3   odo          444 non-null    float64       
 4   end_odo      444 non-null    float64       
 5   soc          441 non-null    float64       
 6   end_soc      443 non-null    float64       
 7   event        444 non-null    category      
 8   charge_mode  444 non-null    category      
dtypes: category(2), datetime64[ns](2), float64(5)
memory usage: 25.6 KB
